In [14]:
import sys
import torch
import numpy as np
from tqdm import tqdm, trange
from model import ConvLSTM
import matplotlib.pyplot as plt
from io import BytesIO
import imageio.v2 as imageio
from matplotlib.colors import Normalize
from utils import TrainDataset, TestDataset

In [15]:
state_path = '/Users/reza/Career/DMLab/SURROGATE/results/laplace/ConvLSTM/pete_sample_1_layer_64_hidden/96.pth'
train_path = '/Users/reza/Career/DMLab/SURROGATE/Data/pete_sample/train'
val_path = '/Users/reza/Career/DMLab/SURROGATE/Data/pete_sample/test'

In [25]:
starting_slice = 100

In [16]:
train_dataset = TrainDataset(train_path)
train_min, train_max = train_dataset.get_min_max()
test_dataset = TestDataset(val_path, train_min, train_max, starting_slice=starting_slice)

In [17]:
hidden_dim, num_layers = 64, 1

In [18]:
model = ConvLSTM(
    input_dim=1,
    hidden_dim=hidden_dim,
    kernel_size=(3, 3),
    num_layers=num_layers,
    batch_first=True,
    bias=True,
    return_all_layers=False,
    output_dim=1
)

In [19]:
model.load_state_dict(torch.load(state_path))

/var/folders/bv/7h0f0hns2y72yqgg3ww8qkkm0000gn/T/ipykernel_1136/1111677253.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(state_path))


<All keys matched successfully>

# Validation partition

In [20]:
for x, y in test_dataset:
    with torch.no_grad():
        # slicing
        # print(x.shape)
        yhat, _ = model(x.unsqueeze(0), teacher_forcing=False, seq_len=y.shape[0])

In [21]:
y = test_dataset[0][1].unsqueeze(0)

In [22]:
yhat.shape, y.shape

(torch.Size([1, 39, 1, 128, 128]), torch.Size([1, 39, 1, 128, 128]))

In [23]:
# norm = Normalize(vmin=0.3, vmax=1.0)

In [26]:
frames = []
for step in trange(yhat.shape[1]):
    val = y[0, step, 0]
    pred = yhat[0, step, 0]
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(val, cmap="viridis")
    # plt.contourf(
    #     np.arange(val.shape[0]),
    #     np.arange(val.shape[1]),
    #     val.T,
    #     cmap="viridis",
    #     # norm=norm,
    # )
    plt.title(f"Actual Slice (k={step+1+starting_slice})")
    plt.colorbar()
    # plt.xlabel("I")
    # plt.ylabel("J")

    plt.subplot(1, 2, 2)
    plt.imshow(pred, cmap="viridis")
    # plt.contourf(
    #     np.arange(pred.shape[0]),
    #     np.arange(pred.shape[1]),
    #     pred.T,
    #     cmap="viridis",
    #     # norm=norm,
    # )
    plt.title(f"Predicted Slice (k={step+1+starting_slice})")
    plt.colorbar()
    # plt.xlabel("I")
    # plt.ylabel("J")

    # plt.show()
    buf = BytesIO()
    plt.savefig(buf, format='png')
    frames.append(imageio.imread(buf))
    plt.close()

100%|██████████| 39/39 [00:05<00:00,  7.52it/s]


In [27]:
output_filename = f"plt.mp4"
fps = 10  # Adjust frames per second as needed

# Create video directly from frames in memory
with imageio.get_writer(output_filename, fps=fps) as writer:
    for frame in frames:
        writer.append_data(frame)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1200, 600) to (1200, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
